In [1]:
WEAVIATE_URL="YOUR_WEAVIATE_URL"
WEAVIATE_APIKEY="YOUR_API_KEY"

In [ ]:
import weaviate

batch_size = 20
class_name = "Passage"
class_properties = ['type', 'paper_id', 'cosmos_object_id']
cursor = None
num_of_figures = 0
num_of_objects = 0
num_of_figures_without_comos_id = 0

def get_batch_with_cursor(client, class_name, class_properties, batch_size, cursor=None):
    query = (
        client.query.get(class_name, class_properties)
        .with_additional(["id"])
        .with_limit(batch_size)
    )

    if cursor is not None:
        return query.with_after(cursor).do()
    else:
        return query.do()

source_client = weaviate.Client(
    url=WEAVIATE_URL, 
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_APIKEY)
)

while True:
    # From the SOURCE instance, get the next group of objects
    results = get_batch_with_cursor(source_client, class_name, class_properties, batch_size, cursor=cursor)

    # If empty, we're finished
    if len(results["data"]["Get"][class_name]) == 0:
        break

    # Otherwise, find the uuid of the figure object and delete it
    objects_list = results["data"]["Get"][class_name]
    num_of_objects += len(objects_list)
    for object in objects_list:
        if object['type'] == 'figure':
            num_of_figures += 1
            if object['cosmos_object_id'] is None:
                num_of_figures_without_comos_id += 1
                print(object)
                print(object['_additional']['id'])
                
                # try:
                #     source_client.data_object.delete(
                #         uuid=object['_additional']['id'],
                #         class_name=class_name,
                #     )
                # except weaviate.exceptions.UnexpectedStatusCodeException as e:
                #     print(e)

    # Update the cursor to the id of the last retrieved object
    cursor = results["data"]["Get"][class_name][-1]["_additional"]["id"]
    
print(f"Total number of objects: {num_of_objects}")
print(f"Total number of figures: {num_of_figures}")
print(f"Total number of figures without cosmos_object_id: {num_of_figures_without_comos_id}")